
# 콘텐츠분쟁해결 RAG 시스템 — 단계별 실습 노트북 (.env 연동판)
**교과 과제:** _문제 3-1 : 콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드_  
**데드라인 안내:** 

> [문제3-1]은 **9월 29일(월) 수업 시작 전**까지 **이메일로 Git 경로**를 제출하세요.  
> (본 노트북은 .env 설정을 자동 반영하며, Git 경로를 기록/검증하는 셀을 포함합니다.)

**자료:** `콘텐츠분쟁해결_사례.pdf` (콘텐츠분쟁해결 사례집)  
**원본 위치(참고):** https://github.com/vega2kgkube/MyLangChainProject/tree/main/src/mylangchain_app/data

---

이 노트북은 **.env** 설정(예: `UPSTAGE_API_KEY`, `RAG_VECTORSTORE`, `RAG_TOP_K`, `UPSTAGE_MODEL` 등)을 자동으로 불러와 아래 **0~8단계**를 순서대로 실습할 수 있게 구성되어 있습니다.

- 0단계: 문서 로드
- 1단계: 문서 분할 설정
- 2단계: 임베딩 모델 설정 (Upstage)
- 3단계: 검색기 설정 (FAISS/Chroma 선택 가능)
- 4단계: LLM 설정 (Upstage `solar-pro`)
- 5단계: 법률 자문 프롬프트 작성
- 6단계: QA 체인 생성 (출처 반환)
- 7단계: 테스트 질문 실행
- 8단계: (선택) 분쟁 유형 분류기

> **주의:** 이 노트북은 키를 직접 출력하지 않습니다. 키 존재 여부만 확인합니다.



## 0-0) 설치(필요 시) & 노트북 버전 정보


In [ ]:

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())


Python: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-11-10.0.26100-SP0



## 0-1) .env 로딩 & 환경변수 보정
- `.env`를 로드하고, 잘못된 키명(예: `UpStage_API_KEY`)을 **`UPSTAGE_API_KEY`** 로 자동 보정합니다.
- 키 값 자체는 출력하지 않고, **설정 여부만 표기**합니다.


In [2]:

from dotenv import load_dotenv
import os

load_dotenv()

# --- env alias normalizer ---
aliases = {
    "UpStage_API_KEY": "UPSTAGE_API_KEY",
    "Upstage_API_KEY": "UPSTAGE_API_KEY",
    "UPSTAGE_api_key": "UPSTAGE_API_KEY",
}
for bad, good in aliases.items():
    if os.getenv(bad) and not os.getenv(good):
        os.environ[good] = os.getenv(bad)

def env_set(name: str) -> str:
    return "set" if bool(os.getenv(name)) else "missing"

check = {
    "UPSTAGE_API_KEY": env_set("UPSTAGE_API_KEY"),
    "OPENAI_API_KEY": env_set("OPENAI_API_KEY"),
    "LANGSMITH_API_KEY": env_set("LANGSMITH_API_KEY"),
    "GOOGLE_API_KEY": env_set("GOOGLE_API_KEY"),
    "TAVILY_API_KEY": env_set("TAVILY_API_KEY"),
}
print("[ENV CHECK]", check)

# RAG 설정값 (키는 아님)
RAG_VECTORSTORE = os.getenv("RAG_VECTORSTORE", "faiss").lower()  # faiss | chroma
RAG_TOP_K = int(os.getenv("RAG_TOP_K", "5"))
UPSTAGE_MODEL = os.getenv("UPSTAGE_MODEL", "solar-pro")
UPSTAGE_EMBEDDING_MODEL = os.getenv("UPSTAGE_EMBEDDING_MODEL", "solar-embedding-1-large")
UPSTAGE_TEMPERATURE = float(os.getenv("UPSTAGE_TEMPERATURE", "0.2"))

print({
    "RAG_VECTORSTORE": RAG_VECTORSTORE,
    "RAG_TOP_K": RAG_TOP_K,
    "UPSTAGE_MODEL": UPSTAGE_MODEL,
    "UPSTAGE_EMBEDDING_MODEL": UPSTAGE_EMBEDDING_MODEL,
    "UPSTAGE_TEMPERATURE": UPSTAGE_TEMPERATURE,
})


[ENV CHECK] {'UPSTAGE_API_KEY': 'set', 'OPENAI_API_KEY': 'set', 'LANGSMITH_API_KEY': 'set', 'GOOGLE_API_KEY': 'set', 'TAVILY_API_KEY': 'set'}
{'RAG_VECTORSTORE': 'faiss', 'RAG_TOP_K': 5, 'UPSTAGE_MODEL': 'solar-pro', 'UPSTAGE_EMBEDDING_MODEL': 'solar-embedding-1-large', 'UPSTAGE_TEMPERATURE': 0.2}



## 0-2) Git 경로 기록(제출용)
수업 제출용 Git 저장소 URL을 여기에 기록하고, 마지막에 다시 출력해 확인합니다.


In [ ]:

GIT_REPO_URL = ""  # 예: "https://github.com/username/mylangchain-app"
print("현재 설정된 제출 Git 경로:", GIT_REPO_URL or "(미설정)")



## 0-3) 데이터 경로 설정 & PDF 자동 탐색


In [ ]:

from pathlib import Path

CANDIDATES = [
    Path.cwd().parent / "src" / "mylangchain_app" / "data" / "콘텐츠분쟁해결_사례.pdf",
]

PDF_PATH = None
for p in CANDIDATES:
    if p.exists():
        PDF_PATH = p
        break

print("PDF_PATH:", str(PDF_PATH) if PDF_PATH else "(찾지 못함)")
if PDF_PATH is None:
    print("⚠️ PDF를 위 경로 중 하나에 위치시키고 다시 실행하세요.")


PDF_PATH: c:\mylangchain\mylangchain-app\src\mylangchain_app\data\콘텐츠분쟁해결_사례.pdf



## 0단계: 문서 로드


In [4]:

from langchain_community.document_loaders import PyPDFLoader

assert PDF_PATH is not None, "PDF를 찾지 못했습니다. 앞 셀에서 경로를 확인하세요."
loader = PyPDFLoader(str(PDF_PATH))
docs = loader.load()
print("Loaded pages:", len(docs))
print("Sample metadata:", docs[0].metadata if docs else {})


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pages: 109
Sample metadata: {'producer': 'Hancom PDF 1.3.0.410', 'creator': 'Hancom PDF 1.3.0.410', 'creationdate': '2011-01-20T18:01:33+09:00', 'title': '제 2절 영국사례', 'moddate': '2011-01-20T18:01:33+09:00', 'pdfversion': '1.4', 'source': 'c:\\mylangchain\\mylangchain-app\\src\\mylangchain_app\\data\\콘텐츠분쟁해결_사례.pdf', 'total_pages': 109, 'page': 0, 'page_label': '1'}



## 1단계: 문서 분할 설정


In [5]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ],
)

splits = text_splitter.split_documents(docs)
print("Total chunks:", len(splits))
print("First chunk preview:\n", (splits[0].page_content[:500] + "…") if splits else "No chunks")


Total chunks: 104
First chunk preview:
 콘텐츠분쟁조정 법리 연구 2부
- 타 분쟁조정사례 조사 -…



## 2단계: 임베딩 모델 설정


In [6]:

from langchain_upstage import UpstageEmbeddings
import os

assert os.getenv("UPSTAGE_API_KEY"), "UPSTAGE_API_KEY가 설정되어 있어야 합니다 (.env 확인)."
embeddings = UpstageEmbeddings(model=os.getenv("UPSTAGE_EMBEDDING_MODEL", "solar-embedding-1-large"))
print("Embeddings ready:", embeddings)


Embeddings ready: client=<openai.resources.embeddings.Embeddings object at 0x00000218C016F170> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000218C041FDA0> model='solar-embedding-1-large' dimensions=None upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1/solar' embedding_ctx_length=4096 embed_batch_size=10 allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers={'x-upstage-client': 'langchain'} default_query=None http_client=None http_async_client=None



## 3단계: 검색기 설정

In [7]:

from langchain_community.vectorstores import FAISS, Chroma
from chromadb.config import Settings as ChromaSettings

INDEX_ROOT = Path.cwd() / ".rag_index"
INDEX_ROOT.mkdir(parents=True, exist_ok=True)

if RAG_VECTORSTORE == "faiss":
    faiss_dir = INDEX_ROOT / "faiss"
    faiss_dir.mkdir(parents=True, exist_ok=True)
    if (faiss_dir / "index.faiss").exists() and (faiss_dir / "index.pkl").exists():
        vectorstore = FAISS.load_local(
            folder_path=str(faiss_dir),
            embeddings=embeddings,
            allow_dangerous_deserialization=True,
        )
        print("Loaded FAISS index from:", faiss_dir)
    else:
        vectorstore = FAISS.from_documents(splits, embeddings)
        vectorstore.save_local(str(faiss_dir))
        print("Built FAISS index and saved to:", faiss_dir)
elif RAG_VECTORSTORE == "chroma":
    chroma_dir = INDEX_ROOT / "chroma"
    chroma_dir.mkdir(parents=True, exist_ok=True)
    collection_name = "legal_cases"
    if not (chroma_dir / "chroma.sqlite3").exists():
        vectorstore = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            persist_directory=str(chroma_dir),
            collection_name=collection_name,
            client_settings=ChromaSettings(anonymized_telemetry=False),
        )
        vectorstore.persist()
        print("Built Chroma index at:", chroma_dir)
    else:
        vectorstore = Chroma(
            embedding_function=embeddings,
            persist_directory=str(chroma_dir),
            collection_name=collection_name,
            client_settings=ChromaSettings(anonymized_telemetry=False),
        )
        print("Loaded Chroma index from:", chroma_dir)
else:
    raise ValueError("RAG_VECTORSTORE must be 'faiss' or 'chroma'")
    
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 다양성 고려 시 "mmr"
    search_kwargs={"k": RAG_TOP_K},
)
print("Retriever ready. (type=", RAG_VECTORSTORE, ")")


Built FAISS index and saved to: c:\mylangchain\mylangchain-app\src\mylangchain_app\0example\.rag_index\faiss
Retriever ready. (type= faiss )



## 4단계: LLM 설정


In [8]:

from langchain_upstage import ChatUpstage

llm = ChatUpstage(
    model=UPSTAGE_MODEL,
    base_url="https://api.upstage.ai/v1",
    temperature=UPSTAGE_TEMPERATURE,
)
print("LLM ready:", UPSTAGE_MODEL, "temp=", UPSTAGE_TEMPERATURE)


LLM ready: solar-pro temp= 0.2



## 5단계: 법률 자문 프롬프트 작성


In [9]:

from langchain.prompts import PromptTemplate

prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
""".strip()

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)
print("Prompt ready.")


Prompt ready.



## 6단계: QA 체인 생성 (출처 반환)


In [10]:

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("QA chain ready.")


QA chain ready.



## 7단계: 테스트 질문 실행
문제에서 제시한 5가지 질문을 사용합니다. (상위 3개 출처를 함께 출력)


In [11]:

TEST_QUESTIONS = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?",
]

def pretty_sources(source_documents, top_k=3):
    from pathlib import Path
    out = []
    for i, d in enumerate(source_documents[:top_k], start=1):
        meta = d.metadata or {}
        page = meta.get("page", meta.get("loc", None))
        src = meta.get("source", "문서")
        snippet = (d.page_content[:200] + "…") if len(d.page_content) > 200 else d.page_content
        out.append(f"  - ({i}) {Path(str(src)).name} p.{page} | {snippet}")
    return "\n".join(out) if out else "  (no sources)"

for q in TEST_QUESTIONS:
    print("\n[질문]", q)
    result = qa_chain({"query": q})
    print("\n[답변]\n", result.get("result", "").strip())
    print("\n[참조 상위 3개]\n", pretty_sources(result.get("source_documents", []), top_k=3))



[질문] 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?


C:\Users\user\AppData\Local\Temp\ipykernel_20156\2719716376.py:22: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": q})



[답변]
 ### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안

#### 1. **사례 기반 분석 및 법적 근거**
   - **사례 1 (2006년 시스템 오류)**  
     - **쟁점**: 계정 명의 불일치 시 복구 거부 가능성  
     - **결과**: 게임사가 계정 공유/현금거래를 금지한 약관을 근거로 복구 거부.  
     - **시사점**: 계정 명의가 본인인지 확인이 필수적이며, 타인 명의 계정 사용 시 법적 보호를 받기 어려움.  
     - **관련 법령**: 「전자상거래 등에서의 소비자보호에 관한 법률」 제14조(계약체결 시 정보제공) 및 게임사 이용약관.

   - **사례 2 (2009년 시스템 오류)**  
     - **쟁점**: 시스템 오류 입증 부족  
     - **결과**: 게임사가 서버 로그 등을 근거로 오류 없음을 주장, 복구 거부.  
     - **시사점**: 시스템 오류 발생 사실을 입증할 증거(스크린샷, 로그 기록 등)가 필요.  
     - **관련 법령**: 「민법」 제390조(채무불이행과 손해배상) - 게임사의 서비스 제공 의무 위반 시 책임 가능성.

   - **사례 3 (2006년 프로그램 오류)**  
     - **쟁점**: 오류 인정 후 복구  
     - **결과**: 게임사가 오류를 인정하고 아이템 복구.  
     - **시사점**: 게임사가 오류를 시인할 경우 복구 가능성 높음.  
     - **관련 법령**: 「소비자기본법」 제16조(결함에 대한 시정 요구).

   - **사례 4 (2007년 해킹 아이템 회수)**  
     - **쟁점**: 불법 아이템 회수 정당성  
     - **결과**: 게임사가 이용약관에 따라 해킹 아이템 회수 후 게임머니 환급.  
     - **시사점**: 아이템의 합법적 취득 여부(해킹/유실물)가 복구 여부에 영향.  
     - **관련 법령**: 「민법」 제250조(도품·유실물 반


## 8단계: 분쟁 유형 분류 함수(선택)


In [12]:

def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    q = (query or "").lower()
    if any(k.lower() in q for k in game_keywords):
        return "게임"
    elif any(k.lower() in q for k in elearning_keywords):
        return "이러닝"
    elif any(k.lower() in q for k in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

def ask_with_type_hint(question: str, top_k_sources: int = 3):
    dtype = classify_dispute_type(question)
    aug_q = f"[분쟁유형:{dtype}] {question}" if dtype != "기타" else question
    result = qa_chain({"query": aug_q})
    print("[분류]", dtype)
    print("\n[답변]\n", result.get("result", "").strip())
    print("\n[참조 상위 3개]\n", pretty_sources(result.get("source_documents", []), top_k=top_k_sources))

# 예시 실행
ask_with_type_hint("무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?")


[분류] 이러닝

[답변]
 ### 전문 법률 조언: 무료체험 후 자동 유료전환 및 환불 요건 분석

#### 1. **사례 근거 및 쟁점 분석**
제시된 사례(한국소비자원 2010_무료 이벤트 후 자동 소액 결제 요금 환급요구)와 유사하며, 다음과 같은 쟁점이 적용됩니다:
- **중요사항 고지의무 위반 여부**: 무료체험 기간 종료 후 자동 유료전환 사실을 소비자가 명확히 인지할 수 있도록 고지했는지 여부.
- **소비자 동의의 유효성**: 소액결제 인증 등 형식적 동의가 실질적 동의로 인정되는지 여부.
- **환급 범위**: 사업자의 책임 비율에 따른 부분 환급 가능성.

#### 2. **관련 법령**
- **약관의 규제에 관한 법률 제3조(약관 작성 및 설명의무)**:  
  사업자는 계약 체결 시 중요한 내용(유료전환 조건 등)을 소비자가 이해할 수 있도록 명시·설명해야 합니다.  
  - *위반 시*: 해당 약관 조항은 무효이며, 소비자는 계약 취소 또는 손해배상을 청구할 수 있습니다.
- **소비자기본법 제18조(소비자분쟁해결기준)**:  
  인터넷콘텐츠업 분야의 경우, 무료체험 후 유료전환 시 **사전 고지 없이 자동 결제된 금액은 전액 환급**이 원칙입니다.  
  - *예외*: 소비자가 고지 내용을 확인했음을 증명하거나, 이용실적에 따라 부분 환급 가능.

#### 3. **유사 사례 처리경위 및 결과**
- **한국소비자원 2010년 사례**:  
  - 사업자가 이벤트 화면에 유료전환 사실을 고지했으나, **활자 크기 작음·확인 절차 미흡**으로 소비자 인지 어려움 인정.  
  - **책임 50% 제한**: 4개월간 이의 없이 결제한 점을 고려해 50% 환급 결정.  
- **전자거래분쟁조정위원회 2006년 사례**:  
  - 무료 광고 후 자동 유료전환 시 **표시 불명확성**으로 사업자 책임 인정, 전액 환급 권고.

#### 4. **실무적 해결방안 (단계별)**
**(1) 계약 내용 확인**  
- 이용약관 및 이벤트 페이지에


## (옵션) 검색 다양성: MMR 리트리버
`search_type="mmr"`로 다양한 청크를 섞어 검색하도록 전환합니다.


In [13]:

retriever_mmr = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": RAG_TOP_K})
from langchain.chains import RetrievalQA as _RetrievalQA
qa_chain_mmr = _RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever_mmr,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("MMR retriever ready.")


MMR retriever ready.


In [14]:

q = "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?"
res = qa_chain_mmr({"query": q})
print("[MMR] 답변:\n", res.get("result", "").strip())
print("\n[MMR] 참조 상위 3개:\n", pretty_sources(res.get("source_documents", []), top_k=3))


[MMR] 답변:
 ### 전문 법률 조언: 인터넷 강의 중도 해지 시 과도한 위약금 요구의 적법성 검토  

#### 1. **사례 분석 및 법적 근거**  
제시된 사례(2007~2010년 이러닝 분쟁조정 사례)에 따르면, **중도 해지 시 위약금 공제**는 계약 조건 및 소비자보호 법령에 따라 그 적법성이 판단됩니다. 주요 기준은 다음과 같습니다:  
- **소비자분쟁해결기준(공정위 고시, 인터넷콘텐츠업)**에 따라 환급금이 산정되어야 하며, 사업자의 일방적 약관만으로 과도한 위약금을 부과할 수 없습니다.  
  - 예시: 2008년 사례(온라인통신교육서비스 이용료 환급 요구2)에서 계약서와 달리 실제 지불액을 기준으로 환급금을 산정하고, 위약금이 과도하다고 판단된 경우 조정안이 제시되었습니다.  
- **방문판매 등에 관한 법률 제29조(계약 해지)** 및 **소비자기본법**에 따라, 사업자는 중도 해지 시 **잔여 이용 기간에 상응하는 금액**을 환급해야 합니다.  
- **약관의 규제에 관한 법률 제6조(불공정 약관 무효)**에 따라, "위약금 = 잔여 금액 전액" 또는 "포맷비용 등 불합리한 항목"은 무효될 수 있습니다.  

#### 2. **과도한 위약금 판단 기준**  
- **사례 2008_온라인통신교육서비스 이용료 환급 요구2**에서 피신청인은 위약금·포맷비용 등을 공제했으나, 실제 지불액과 소비자분쟁해결기준을 적용해 환급금이 조정되었습니다.  
- **사례 2008_온라인 동영상 강의 계약 해지 청구**에서는 교재 훼손 여부와 서비스 이용 사실이 청약철회권 제한 사유로 검토되었으나, **이용하지 않은 부분**에 대해서는 부분적 환불이 인정되었습니다.  

#### 3. **실무적 해결 방안**  
**(1) 계약 조건 확인**  
- 계약서 또는 약관에서 **중도 해지 시 환급 기준**을 확인합니다.  
  - "이용 기간 비율"에 따른 환급 규정이 있는지, 위약금 항목이 명시되어 있는지 검토합니다.  
- **청약철회 기간(7일 이내)